In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dropout
from keras.activations import relu
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import MobileNetV2

In [8]:
!pip install kaggle


In [9]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [10]:
!mkdir -p ~/.kaggle


In [11]:
!cp kaggle.json ~/.kaggle/


In [12]:
!chmod 600 /root/.kaggle/kaggle.json


In [13]:
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid
!unzip -q monkeypox-skin-images-dataset-msid

 81% 44.0M/54.6M [00:00<00:00, 83.0MB/s]
100% 54.6M/54.6M [00:00<00:00, 77.3MB/s]


In [14]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [16]:
baseModel = MobileNetV2(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
drpOut1 = Dropout(0.5)(dense_1)
dense2 = Dense(256, activation='relu')(drpOut1)
drpout2 = Dropout(0.2)(dense2)
output_layer = Dense(4, activation='softmax')(drpout2)


In [18]:
model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_3 (Flatten)         (None, 62720)             0         
                                                                 
 dense_9 (Dense)             (None, 512)               32113152  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                           

In [19]:
history = model.fit(
    TrainData,
    epochs=50,  # adjust the number of epochs as needed
    validation_data=validationData
)

Epoch 1/50
20/20 [==============================] - 30s 1s/step - loss: 8.4406 - accuracy: 0.5583 - val_loss: 3.6217 - val_accuracy: 0.6118
Epoch 2/50
20/20 [==============================] - 26s 1s/step - loss: 3.6538 - accuracy: 0.7362 - val_loss: 2.4337 - val_accuracy: 0.7500
Epoch 3/50
20/20 [==============================] - 27s 1s/step - loss: 1.9517 - accuracy: 0.8172 - val_loss: 1.2461 - val_accuracy: 0.8421
Epoch 4/50
20/20 [==============================] - 26s 1s/step - loss: 1.2497 - accuracy: 0.8754 - val_loss: 1.2052 - val_accuracy: 0.8553
Epoch 5/50
20/20 [==============================] - 27s 1s/step - loss: 0.8992 - accuracy: 0.9029 - val_loss: 1.4120 - val_accuracy: 0.8684
Epoch 6/50
20/20 [==============================] - 27s 1s/step - loss: 0.5450 - accuracy: 0.9337 - val_loss: 1.0501 - val_accuracy: 0.8684
Epoch 7/50
20/20 [==============================] - 27s 1s/step - loss: 0.5707 - accuracy: 0.9320 - val_loss: 1.0017 - val_accuracy: 0.8289
Epoch 8/50
20/20 [==